![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# ONCOLOGY MODELS

This notebook includes details about different kinds of pretrained models to extract oncology-related information from clinical texts, together with examples of each type of model.

## Setup

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.pretrained import InternalResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp.version : ',sparknlp.version())
print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp.version :  5.1.0
sparknlp_jsl.version :  5.1.0


## List of Pretrained Models

In [0]:
df = pd.DataFrame()
for model_type in ['MedicalNerModel', 'BertForTokenClassification', 'RelationExtractionModel', 'RelationExtractionDLModel', 'AssertionDLModel']:
    model_list = sorted(list(set([model[0] for model in InternalResourceDownloader.returnPrivateModels(model_type) if 'oncology' in model[0]])))
    if len(model_list) > 0:
      if model_type == "MedicalNerModel":
        model_list = list(filter(lambda x: "wip" not in x, model_list))
      df = pd.concat([df, pd.DataFrame(model_list, columns = [model_type])], axis = 1)
    
df.fillna('')

,MedicalNerModel,RelationExtractionModel,RelationExtractionDLModel,AssertionDLModel
0,ner_oncology,re_oncology_biomarker_result_wip,redl_oncology_biobert_wip,assertion_oncology_demographic_binary_wip
1,ner_oncology_anatomy_general,re_oncology_granular_wip,redl_oncology_biomarker_result_biobert_wip,assertion_oncology_family_history_wip
2,ner_oncology_anatomy_general_healthcare,re_oncology_location_wip,redl_oncology_granular_biobert_wip,assertion_oncology_problem_wip
3,ner_oncology_anatomy_general_langtest,re_oncology_size_wip,redl_oncology_location_biobert_wip,assertion_oncology_response_to_treatment_wip
4,ner_oncology_anatomy_granular,re_oncology_temporal_wip,redl_oncology_size_biobert_wip,assertion_oncology_smoking_status_wip
5,ner_oncology_anatomy_granular_langtest,re_oncology_test_result_wip,redl_oncology_temporal_biobert_wip,assertion_oncology_test_binary_wip
6,ner_oncology_biomarker,re_oncology_wip,redl_oncology_test_result_biobert_wip,assertion_oncology_treatment_binary_wip
7,ner_oncology_biomarker_healthcare,,,assertion_oncology_wip
8,ner_oncology_demographics,,,
9,ner_oncology_demographics_langtest,,,


**Medidical NER Models and labels**

<br>


**labels**                 | **description**                                                                                                                                                                                                                        | **ner_oncology** | **ner_oncology_anatomy_general** | **ner_oncology_anatomy_general_healthcare** | **ner_oncology_anatomy_granular** | **ner_oncology_biomarker** | **ner_oncology_biomarker_healthcare** | **ner_oncology_demographics** | **ner_oncology_diagnosis** | **ner_oncology_emb_clinical_large** | **ner_oncology_emb_clinical_medium** | **ner_oncology_limited_80p_for_benchmarks** | **ner_oncology_posology** | **ner_oncology_response_to_treatment** | **ner_oncology_test** | **ner_oncology_therapy** | **ner_oncology_tnm** | **ner_oncology_unspecific_posology** | **ner_oncology_unspecific_posology_healthcare**
:-------------------------|:--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------:|:--------------------------------:|:-------------------------------------------:|:---------------------------------:|:--------------------------:|:-------------------------------------:|:-----------------------------:|:--------------------------:|:-----------------------------------:|:------------------------------------:|:-------------------------------------------:|:-------------------------:|:--------------------------------------:|:---------------------:|:------------------------:|:--------------------:|:------------------------------------:|:-----------------------------------------------:
 **Adenopathy**            | Mentions of pathological findings of the lymph nodes.                                                                                                                                                                                  | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Age**                   | All mention of ages, past or present, related to the patient or with anybody else.                                                                                                                                                     | X                |                                  |                                             |                                   |                            |                                       | X                             |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Biomarker**             | Biological molecules that indicate the presence or absence of cancer, or the type of cancer. Oncogenes are excluded from this category.                                                                                                | X                |                                  |                                             |                                   | X                          | X                                     |                               |                            | X                                   | X                                    | X                                           |                           |                                        | X                     |                          |                      |                                      |                                                 
 **Biomarker_Result**      | Terms or values that are identified as the result of a biomarkers.                                                                                                                                                                     | X                |                                  |                                             |                                   | X                          | X                                     |                               |                            | X                                   | X                                    | X                                           |                           |                                        | X                     |                          |                      |                                      |                                                 
 **Cancer_Dx**             | Mentions of cancer diagnoses (such as “breast cancer”) or pathological types that are usually used as synonyms for “cancer” (e.g. “carcinoma”). <br> When anatomical references are present, they are included in the Cancer_Dx extraction. | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          | X                    |                                      |                                                 
 **Cancer_Score**          | Clinical or imaging scores that are specific for cancer settings (e.g. “BI-RADS” or “Allred score”).                                                                                                                                   | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Cancer_Surgery**        | Terms that indicate surgery as a form of cancer treatment.                                                                                                                                                                             | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Chemotherapy**          | Mentions of chemotherapy drugs, or unspecific words such as “chemotherapy”.                                                                                                                                                            | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       | X                        |                      |                                      |                                                 
 **Cycle_Count**           | The total number of cycles being administered of an oncological therapy (e.g. “5 cycles”).                                                                                                                                             | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Cycle_Day**             | References to the day of the cycle of oncological therapy (e.g. “day 5”).                                                                                                                                                              | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Cycle_Number**          | The number of the cycle of an oncological therapy that is being applied (e.g. “third cycle”).                                                                                                                                          | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Date**                  | Mentions of exact dates, in any format, including day number, month and/or year.                                                                                                                                                       | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Death_Entity**          | Words that indicate the death of the patient or someone else (including family members), such as “died” or “passed away”.                                                                                                              | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Direction**             | Directional and laterality terms, such as “left”, “right”, “bilateral”, “upper” and “lower”.                                                                                                                                           | X                | X                                | X                                           | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Dosage**                | The quantity prescribed by the physician for an active ingredient.                                                                                                                                                                     | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Duration**              | Words indicating the duration of a treatment (e.g. “for 2 weeks”).                                                                                                                                                                     | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Frequency**             | Words indicating the frequency of treatment administration (e.g. “daily” or “bid”).                                                                                                                                                    | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Gender**                | Gender-specific nouns and pronouns (including words such as “him” or “she”, and family members such as “father”).                                                                                                                      | X                |                                  |                                             |                                   |                            |                                       | X                             |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Grade**                 | All pathological grading of tumors (e.g. “grade 1”) or degrees of cellular differentiation (e.g. “well-differentiated”)                                                                                                                | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Histological_Type**     | Histological variants or cancer subtypes, such as “papillary”, “clear cell” or “medullary”.                                                                                                                                            | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Hormonal_Therapy**      | Mentions of hormonal drugs used to treat cancer, or unspecific words such as “hormonal therapy”.                                                                                                                                       | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       | X                        |                      |                                      |                                                 
 **Imaging_Test**          | Imaging tests mentioned in texts, such as “chest CT scan”.                                                                                                                                                                             | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        | X                     |                          |                      |                                      |                                                 
 **Immunotherapy**         | Mentions of immunotherapy drugs, or unspecific words such as “immunotherapy”.                                                                                                                                                          | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       | X                        |                      |                                      |                                                 
 **Invasion**              | Mentions that refer to tumor invasion, such as “invasion” or “involvement”. Metastases or lymph node involvement are excluded from this category.                                                                                      | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Line_Of_Therapy**       | Explicit references to the line of therapy of an oncological therapy (e.g. “first-line treatment”).                                                                                                                                    | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           | X                                      |                       | X                        |                      |                                      |                                                 
 **Metastasis**            | Terms that indicate a metastatic disease. Anatomical references are not included in these extractions.                                                                                                                                 | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          | X                    |                                      |                                                 
 **Oncogene**              | Mentions of genes that are implicated in the etiology of cancer.                                                                                                                                                                       | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        | X                     |                          |                      |                                      |                                                 
 **Pathology_Result**      | The findings of a biopsy from the pathology report that is not covered by another entity (e.g. “malignant ductal cells”).                                                                                                              | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Pathology_Test**        | Mentions of biopsies or tests that use tissue samples.                                                                                                                                                                                 | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        | X                     |                          |                      |                                      |                                                 
 **Performance_Status**    | Mentions of performance status scores, such as ECOG and Karnofsky. The name of the score is extracted together with the result (e.g. “ECOG performance status of 4”).                                                                  | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Race_Ethnicity**        | The race and ethnicity categories include racial and national origin or sociocultural groups.                                                                                                                                          | X                |                                  |                                             |                                   |                            |                                       | X                             |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Radiotherapy**          | Terms that indicate the use of Radiotherapy.                                                                                                                                                                                           | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Response_To_Treatment** | Terms related to clinical progress of the patient related to cancer treatment, including “recurrence”, “bad response” or “improvement”.                                                                                                | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           | X                                      |                       | X                        |                      |                                      |                                                 
 **Relative_Date**         | Temporal references that are relative to the date of the text or to any other specific date (e.g. “yesterday” or “three years later”).                                                                                                 | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Route**                 | Words indicating the type of administration route (such as “PO” or “transdermal”).                                                                                                                                                     | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Site_Bone**             | Anatomical terms that refer to the human skeleton.                                                                                                                                                                                     | X                |                                  |                                             | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Site_Brain**            | Anatomical terms that refer to the central nervous system (including the brain stem and the cerebellum).                                                                                                                               | X                |                                  |                                             | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Site_Breast**           | Anatomical terms that refer to the breasts.                                                                                                                                                                                            | X                |                                  |                                             | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Site_Liver**            | Anatomical terms that refer to the liver.                                                                                                                                                                                              | X                |                                  |                                             | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Site_Lung**             | Anatomical terms that refer to the lungs.                                                                                                                                                                                              | X                |                                  |                                             | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Site_Lymph_Node**       | Anatomical terms that refer to lymph nodes, excluding adenopathies.                                                                                                                                                                    | X                |                                  |                                             | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Site_Other_Body_Part**  | Relevant anatomical terms that are not included in the rest of the anatomical entities.                                                                                                                                                | X                |                                  |                                             | X                                 |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Smoking_Status**        | All mentions of smoking related to the patient or to someone else.                                                                                                                                                                     | X                |                                  |                                             |                                   |                            |                                       | X                             |                            | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Staging**               | Mentions of cancer stage such as “stage 2b” or “T2N1M0”. It also includes words such as “in situ”, “early-stage” or “advanced”.                                                                                                        | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          | X                    |                                      |                                                 
 **Targeted_Therapy**      | Mentions of targeted therapy drugs, or unspecific words such as “targeted therapy”.                                                                                                                                                    | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       | X                        |                      |                                      |                                                 
 **Tumor_Finding**         | All nonspecific terms that may be related to tumors, either malignant or benign (for example: “mass”, “tumor”, “lesion”, or “neoplasm”).                                                                                               | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Tumor_Size**            | Size of the tumor, including numerical value and unit of measurement (e.g. “3 cm”).                                                                                                                                                    | X                |                                  |                                             |                                   |                            |                                       |                               | X                          | X                                   | X                                    | X                                           |                           |                                        |                       |                          |                      |                                      |                                                 
 **Unspecific_Therapy**    | Terms that indicate a known cancer therapy but that is not specific to any other therapy entity (e.g. “chemoradiotherapy” or “adjuvant therapy”).                                                                                      | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           |                           |                                        |                       | X                        |                      |                                      |                                                 
 **Radiation_Dose**        | Dose used in radiotherapy.                                                                                                                                                                                                             | X                |                                  |                                             |                                   |                            |                                       |                               |                            | X                                   | X                                    | X                                           | X                         |                                        |                       | X                        |                      |                                      |                                                 
 **Anatomical_Site**       | Relevant anatomical terms mentioned in text.                                                                                                                                                                                           |                  | X                                | X                                           |                                   |                            |                                       |                               |                            |                                     |                                      |                                             |                           |                                        |                       |                          |                      |                                      |                                                 
 **Cancer_Therapy**        | Mentions of cancer treatments, including chemotherapy, radiotherapy, surgery and other.                                                                                                                                                |                  |                                  |                                             |                                   |                            |                                       |                               |                            |                                     |                                      |                                             | X                         |                                        |                       |                          |                      | X                                    | X                                               
 **Size_Trend**            | Terms related to the changes in the size of the tumor (such as “growth” or “reduced in size”).                                                                                                                                         |                  |                                  |                                             |                                   |                            |                                       |                               |                            |                                     |                                      |                                             |                           | X                                      |                       |                          |                      |                                      |                                                 
 **Lymph_Node**            | Mentions of lymph nodes and pathological findings of the lymph nodes.                                                                                                                                                                  |                  |                                  |                                             |                                   |                            |                                       |                               |                            |                                     |                                      |                                             |                           |                                        |                       |                          | X                    |                                      |                                                 
 **Tumor_Description**     | Information related to tumor characteristics, such as size, presence of invasion, grade and hystological type.                                                                                                                         |                  |                                  |                                             |                                   |                            |                                       |                               |                            |                                     |                                      |                                             |                           |                                        |                       |                          | X                    |                                      |                                                 
 **Tumor**                 | All nonspecific terms that may be related to tumors, either malignant or benign (for example: “mass”, “tumor”, “lesion”, or “neoplasm”).                                                                                               |                  |                                  |                                             |                                   |                            |                                       |                               |                            |                                     |                                      |                                             |                           |                                        |                       |                          | X                    |                                      |                                                 
 **Lymph_Node_Modifier**   | Words that refer to a lymph node being abnormal (such as “enlargement”).                                                                                                                                                               |                  |                                  |                                             |                                   |                            |                                       |                               |                            |                                     |                                      |                                             |                           |                                        |                       |                          | X                    |                                      |                                                 
 **Posology_Information**  | Terms related to the posology of the treatment, including duration, frequencies and dosage.                                                                                                                                            |                  |                                  |                                             |                                   |                            |                                       |                               |                            |                                     |                                      |                                             |                           |                                        |                       |                          |                      | X                                    | X

## NER Models

The NER models from the list include different entity groups and levels of granularity. If you want to extract as much information as possible from oncology texts, then ner_oncology is the best option for you, as it is the most general and granular model. But you may want to use other models depending on your needs (for instance, if you need to extract information related with staging, ner_oncology_tnm would be the most suitable model).

In [0]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
        
sentence_detector = SentenceDetectorDLModel.pretrained("sentence_detector_dl_healthcare","en","clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence")
 
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")\
    .setSplitChars(["-", "\/"])

word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
    .setInputCols(["sentence","token"])\
    .setOutputCol("embeddings")

# ner_oncology

ner_oncology = MedicalNerModel.pretrained("ner_oncology","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology")\

ner_oncology_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner_oncology"])\
    .setOutputCol("ner_oncology_chunk")

# ner_oncology_tnm

ner_oncology_tnm = MedicalNerModel.pretrained("ner_oncology_tnm","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_tnm")\

ner_oncology_tnm_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner_oncology_tnm"])\
    .setOutputCol("ner_oncology_tnm_chunk")

# # ner_oncology_biomarker

ner_oncology_biomarker = MedicalNerModel.pretrained("ner_oncology_biomarker","en","clinical/models")\
    .setInputCols(["sentence","token","embeddings"])\
    .setOutputCol("ner_oncology_biomarker")\

ner_oncology_biomarker_converter = NerConverterInternal()\
    .setInputCols(["sentence","token","ner_oncology_biomarker"])\
    .setOutputCol("ner_oncology_biomarker_chunk")

ner_stages = [document_assembler,
    sentence_detector,
    tokenizer,
    word_embeddings,
    ner_oncology,
    ner_oncology_converter,
    ner_oncology_tnm,
    ner_oncology_tnm_converter,
    ner_oncology_biomarker,
    ner_oncology_biomarker_converter]

ner_pipeline = Pipeline(stages=ner_stages)

empty_data = spark.createDataFrame([[""]]).toDF("text")

ner_model = ner_pipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 367.3 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_oncology download started this may take some time.
[OK!]
ner_oncology_tnm download started this may take some time.
[OK!]
ner_oncology_biomarker download started this may take some time.
[OK!]


In [0]:
ner_oncology_labels = sorted(list(set([label.split('-')[-1] for label in ner_oncology.getClasses() if label != 'O']))) 

len(ner_oncology_labels)

49

In [0]:
label_df = pd.DataFrame()
for column in range((len(ner_oncology_labels)//10)+1):
  label_df = pd.concat([label_df, pd.DataFrame(ner_oncology_labels, columns = [''])[column*10:(column+1)*10].reset_index(drop= True)], axis = 1)

label_df.fillna('')

,,,,,
0,Adenopathy,Cycle_Number,Hormonal_Therapy,Race_Ethnicity,Site_Lung
1,Age,Date,Imaging_Test,Radiation_Dose,Site_Lymph_Node
2,Biomarker,Death_Entity,Immunotherapy,Radiotherapy,Site_Other_Body_Part
3,Biomarker_Result,Direction,Invasion,Relative_Date,Smoking_Status
4,Cancer_Dx,Dosage,Line_Of_Therapy,Response_To_Treatment,Staging
5,Cancer_Score,Duration,Metastasis,Route,Targeted_Therapy
6,Cancer_Surgery,Frequency,Oncogene,Site_Bone,Tumor_Finding
7,Chemotherapy,Gender,Pathology_Result,Site_Brain,Tumor_Size
8,Cycle_Count,Grade,Pathology_Test,Site_Breast,Unspecific_Therapy
9,Cycle_Day,Histological_Type,Performance_Status,Site_Liver,


In [0]:
ner_oncology_tnm_labels = list(set([label.split('-')[-1] for label in ner_oncology_tnm.getClasses() if label != 'O']))

print(ner_oncology_tnm_labels)

['Tumor_Description', 'Cancer_Dx', 'Tumor', 'Metastasis', 'Lymph_Node', 'Staging', 'Lymph_Node_Modifier']


In [0]:
ner_oncology_biomarker_labels = list(set([label.split('-')[-1] for label in ner_oncology_biomarker.getClasses() if label != 'O']))

print(ner_oncology_biomarker_labels)

['Biomarker_Result', 'Biomarker']


In [0]:
sample_text_1 = '''A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total anterior hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma. Neoadjuvant chemotherapy with the regimens of Taxotere (75 mg/m2), Epirubicin (75 mg/m2), and Cyclophosphamide (500 mg/m2) was given for 6 cycles with poor response, followed by a modified radical mastectomy (MRM) with dissection of axillary lymph nodes and skin grafting. Postoperatively, radiotherapy was done with 5000 cGy in 25 fractions. The histopathologic examination revealed a metaplastic carcinoma with squamous differentiation associated with adenomyoepithelioma. Immunohistochemistry study showed that the tumor cells are positive for epithelial markers-cytokeratin (AE1/AE3) stain, and myoepithelial markers, including cytokeratin 5/6 (CK 5/6), p63, and S100 stains. Expressions of hormone receptors, including ER, PR, and Her-2/Neu, were all negative. The dissected axillary lymph nodes showed metastastic carcinoma with negative hormone receptors in 3 nodes. The patient was staged as pT3N1aM0, with histologic tumor grade III.'''

sample_text_2 = '''She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass. A Pap smear performed one month later was positive for atypical glandular cells suspicious for adenocarcinoma. The pathologic specimen showed extension of the tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged lymph nodes. The final pathologic diagnosis of the tumor was stage IIIC papillary serous ovarian adenocarcinoma. Two months later, the patient was diagnosed with lung metastases.'''

sample_text_3 = '''In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive for CD9, CD10, CD13, CD19, CD20, CD34, CD38, CD58, CD66c, CD123, HLA-DR, cCD79a, and TdT on flow cytometry.

Measurements of serum tumor markers showed elevated level of cytokeratin 19 fragment (Cyfra21-1: 4.77 ng/mL), neuron-specific enolase (NSE: 19.60 ng/mL), and squamous cell carcinoma antigen (SCCA: 2.58 ng/mL). The results were negative for serum carbohydrate antigen 125 (CA125), carcinoembryonic antigen (CEA) and vascular endothelial growth factor (VEGF). Immunohistochemical staining showed positive staining for CK5/6, P40 and PD-L1 (+ 80% tumor cells), and negative staining for TTF-1, PD-1 and weakly positive staining for ALK. Molecular analysis indicated no EGFR mutation or ROS1 fusion.'''

In [0]:
data = spark.createDataFrame(pd.DataFrame([sample_text_1, sample_text_2, sample_text_3], columns = ['text']))

In [0]:
results = ner_model.transform(data).collect()

In [0]:
from sparknlp_display import NerVisualizer

visualiser = NerVisualizer()

In [0]:
ner_html_1 = visualiser.display(results[0], label_col='ner_oncology_chunk', return_html=True)

displayHTML(ner_html_1)

A 65-year-old Age woman Gender had a history of debulking surgery Cancer_Surgery , bilateral Direction oophorectomy Cancer_Surgery with omentectomy Cancer_Surgery , total anterior hysterectomy Cancer_Surgery with radical pelvic lymph nodes dissection Cancer_Surgery due to ovarian carcinoma Cancer_Dx ( mucinous-type Histological_Type carcinoma Cancer_Dx , stage Ic Staging ) 1 year ago Relative_Date . Patient's medical compliance was poor and failed to complete her Gender chemotherapy Chemotherapy ( cyclophosphamide Chemotherapy 750 mg/m2 Dosage , carboplatin Chemotherapy 300 mg/m2 Dosage ). Recently, she Gender noted a palpable right Direction breast Site_Breast mass Tumor_Finding , 15 cm Tumor_Size in size which nearly occupied the whole right Direction breast Site_Breast in 2 months Relative_Date . Core needle biopsy Pathology_Test revealed metaplastic Histological_Type carcinoma Cancer_Dx . Neoadjuvant chemotherapy Chemotherapy with the regimens of Taxotere Chemotherapy ( 75 mg/m2 Dosage ), Epirubicin Chemotherapy ( 75 mg/m2 Dosage ), and Cyclophosphamide Chemotherapy ( 500 mg/m2 Dosage ) was given for 6 cycles Cycle_Count with poor response Response_To_Treatment , followed by a modified radical mastectomy Cancer_Surgery ( MRM Cancer_Surgery ) with dissection of axillary lymph nodes Cancer_Surgery and skin grafting Cancer_Surgery . Postoperatively, radiotherapy Radiotherapy was done with 5000 cGy Radiation_Dose in 25 fractions Cycle_Count . The histopathologic examination Pathology_Test revealed a metaplastic Histological_Type carcinoma Cancer_Dx with squamous differentiation Histological_Type associated with adenomyoepithelioma Cancer_Dx . Immunohistochemistry study Pathology_Test showed that the tumor cells Pathology_Result are positive Biomarker_Result for epithelial markers-cytokeratin Biomarker ( AE1/AE3 Biomarker ) stain, and myoepithelial markers Biomarker , including cytokeratin 5/6 Biomarker ( CK 5/6 Biomarker ), p63 Biomarker , and S100 Biomarker stains. Expressions of hormone receptors Biomarker , including ER Biomarker , PR Biomarker , and Her-2/Neu Oncogene , were all negative Biomarker_Result . The dissected axillary lymph nodes Site_Lymph_Node showed metastastic Metastasis carcinoma Cancer_Dx with negative Biomarker_Result hormone receptors Biomarker in 3 nodes Site_Lymph_Node . The patient was staged as pT3N1aM0 Staging , with histologic tumor Tumor_Finding grade III Grade .

In [0]:
ner_html_2 = visualiser.display(results[1], label_col='ner_oncology_tnm_chunk', return_html=True)

displayHTML(ner_html_2)

She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass Tumor . A Pap smear performed one month later was positive for atypical glandular cells Tumor_Description suspicious for adenocarcinoma Cancer_Dx . The pathologic specimen showed extension Tumor_Description of the tumor Tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged Lymph_Node_Modifier lymph nodes Lymph_Node . The final pathologic diagnosis of the tumor Tumor was stage IIIC Staging papillary Tumor_Description serous Tumor_Description ovarian adenocarcinoma Cancer_Dx . Two months later, the patient was diagnosed with lung metastases Metastasis .

In [0]:
ner_html_3 = visualiser.display(results[2], label_col='ner_oncology_biomarker_chunk', return_html=True)

displayHTML(ner_html_3)

In the bone- marrow (BM) aspiration, blasts accounted for 88.1% of ANCs, which were positive Biomarker_Result for CD9 Biomarker , CD10 Biomarker , CD13 Biomarker , CD19 Biomarker , CD20 Biomarker , CD34 Biomarker , CD38 Biomarker , CD58 Biomarker , CD66c Biomarker , CD123 Biomarker , HLA-DR Biomarker , cCD79a Biomarker , and TdT Biomarker on flow cytometry. Measurements of serum tumor markers showed elevated level Biomarker_Result of cytokeratin 19 fragment Biomarker ( Cyfra21-1 Biomarker : 4.77 ng/mL Biomarker_Result ), neuron-specific enolase Biomarker ( NSE Biomarker : 19.60 ng/mL Biomarker_Result ), and squamous cell carcinoma antigen Biomarker ( SCCA Biomarker : 2.58 ng/mL Biomarker_Result ). The results were negative Biomarker_Result for serum carbohydrate antigen 125 Biomarker ( CA125 Biomarker ), carcinoembryonic antigen Biomarker ( CEA Biomarker ) and vascular endothelial growth factor Biomarker ( VEGF Biomarker ). Immunohistochemical staining showed positive Biomarker_Result staining for CK5/6 Biomarker , P40 Biomarker and PD-L1 Biomarker ( + 80% Biomarker_Result tumor cells), and negative Biomarker_Result staining for TTF-1 Biomarker , PD-1 Biomarker and weakly positive Biomarker_Result staining for ALK Biomarker . Molecular analysis indicated no EGFR Biomarker mutation Biomarker_Result or ROS1 Biomarker fusion Biomarker_Result .

## Relation Extraction Models

RE Models are used to link entities that are related. For oncology entities, you can use general models (such as re_oncology_granular_wip) or you can select a specific model depending on your needs (e.g. re_oncology_size_wip to link tumors and their sizes, or re_oncology_biomarker_result_wip to link biomarkers and their results).

In [0]:
pos_tagger = PerceptronModel.pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentence", "token"]) \
    .setOutputCol("pos_tags")

dependency_parser = DependencyParserModel.pretrained("dependency_conllu", "en") \
    .setInputCols(["sentence", "pos_tags", "token"]) \
    .setOutputCol("dependencies")

re_oncology_granular_wip = RelationExtractionModel.pretrained("re_oncology_granular_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_granular_wip") \
    .setRelationPairs(['Date-Cancer_Dx', 'Cancer_Dx-Date', 'Tumor_Finding-Site_Breast', 'Site_Breast-Tumor_Finding',
                       'Relative_Date-Tumor_Finding', 'Tumor_Fiding-Relative_Date', 'Tumor_Finding-Tumor_Size', 'Tumor_Size-Tumor_Finding',
                       'Pathology_Test-Cancer_Dx', 'Cancer_Dx-Pathology_Test']) \
    .setMaxSyntacticDistance(10)    

re_oncology_size_wip = RelationExtractionModel.pretrained("re_oncology_size_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_size_wip") \
    .setRelationPairs(['Tumor_Finding-Tumor_Size', 'Tumor_Size-Tumor_Finding']) \
    .setMaxSyntacticDistance(10)    

re_oncology_biomarker_result_wip = RelationExtractionModel.pretrained("re_oncology_biomarker_result_wip", "en", "clinical/models") \
    .setInputCols(["embeddings", "pos_tags", "ner_oncology_biomarker_chunk", "dependencies"]) \
    .setOutputCol("re_oncology_biomarker_result_wip") \
    .setRelationPairs(['Biomarker-Biomarker_Result', 'Biomarker_Result-Biomarker']) \
    .setMaxSyntacticDistance(10)      

re_stages = ner_stages + [pos_tagger, dependency_parser, re_oncology_granular_wip, re_oncology_size_wip, re_oncology_biomarker_result_wip]

re_pipeline = Pipeline(stages=re_stages)

re_model = re_pipeline.fit(empty_data)

pos_clinical download started this may take some time.
Approximate size to download 1.5 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.7 MB
[OK!]
re_oncology_granular_wip download started this may take some time.
[OK!]
re_oncology_size_wip download started this may take some time.
[OK!]
re_oncology_biomarker_result_wip download started this may take some time.
[OK!]


In [0]:
sample_text_4 = '''Two years ago, she noted a palpable right breast mass, 15 cm in size. Core needle biopsy revealed metaplastic carcinoma.'''

sample_text_5 = '''The patient presented a 2 cm mass in her left breast, and the tumor in her other breast was 3 cm long.'''

sample_text_6 = '''Immunohistochemical staining showed positive staining for CK5/6, P40 and PD-L1, and negative staining for TTF-1, PD-1 and weakly positive staining for ALK. Immunohistochemistry study showed that the tumor cells are positive for epithelial markers-cytokeratin and myoepithelial markers, including cytokeratin 5/6, p63, and S100 stains.'''

In [0]:
re_data = spark.createDataFrame(pd.DataFrame([sample_text_4, sample_text_5, sample_text_6], columns = ['text']))

In [0]:
re_results = re_model.transform(re_data).collect()

In [0]:
from sparknlp_display import RelationExtractionVisualizer

re_visualiser = RelationExtractionVisualizer()

In [0]:
re_html_1 = re_visualiser.display(re_results[0], relation_col='re_oncology_granular_wip', return_html=True)

displayHTML(re_html_1)

Two years ago RELATIVE_DATE , she noted a palpable right breast SITE_BREAST mass TUMOR_FINDING , 15 cm TUMOR_SIZE in size. Core needle biopsy PATHOLOGY_TEST revealed metaplastic carcinoma CANCER_DX . <polyline fill="none" points="528.0,55.0 528.6938775510204,53.79607986468223 529.3877551020408,52.593128713376224 530.0816326530612,51.39211553009376 530.7755102040817,50.194009298846574 531.4693877551022,48.999779003646445 532.1632653061225,47.810393628505125 532.857142857143,46.626822157434425 533.5510204081633,45.450033574446024 534.2448979591837,44.280996863551756 534.9387755102041,43.12068100876335 535.6326530612243,41.97005499409259 536.3265306122448,40.83008780355124 537.0204081632653,39.70174842115105 537.7142857142858,38.586005830903794 538.4081632653063,37.48382901682123 539.1020408163265,36.396186962915124 539.795918367347,35.32404865319723 540.4897959183673,34.26838307167932 541.1836734693878,33.23015920237316 541.8775510204082,32.21034602929051 542.5714285714286,31.209912536443145 543.265306122449,30.229827707842823 543.9591836734694,29.271060527501298 544.6530612244899,28.334579979430348 545.3469387755104,27.421355047641715 546.0408163265306,26.532354716147186 546.7346938775511,25.668547968958514 547.4285714285716,24.83090379008747 548.1224489795918,24.020391163545803 548.8163265306123,23.237979073345294 549.5102040816327,22.484636503497697 550.2040816326531,21.761332438014776 550.8979591836735,21.069035860908297 551.5918367346939,20.408715756190023 552.2857142857143,19.781341107871725 552.9795918367347,19.187880899965155 553.6734693877552,18.629304116482082 554.3673469387755,18.106579741434267 555.0612244897959,17.620676758833483 555.7551020408164,17.172564152691482 556.4489795918367,16.76321090702004 557.1428571428572,16.393586005830905 557.8367346938776,16.064658433135854 558.530612244898,15.77739717294665 559.2244897959184,15.53277120927505 559.9183673469388,15.33174952613282 560.6122448979592,15.175301107531727 561.3061224489796,15.064394937483533 562.0,15.0 562.0,15.0 562.6938775510204,14.982762284422305 563.3877551020407,15.012035801409274 564.0816326530611,15.086851566949145 564.7755102040816,15.20624059703015 565.4693877551019,15.369233907640522 566.1632653061224,15.574862514768505 566.857142857143,15.822157434402333 567.5510204081634,16.11014968253024 568.2448979591837,16.43787027514046 568.938775510204,16.80435022822123 569.6326530612242,17.208620557760792 570.3265306122447,17.64971227974738 571.0204081632653,18.126656410169232 571.7142857142857,18.638483965014576 572.4081632653061,19.184225960271654 573.1020408163265,19.762913411928704 573.7959183673471,20.37357733597396 574.4897959183673,21.01524874839565 575.1836734693877,21.686958665182026 575.8775510204081,22.387738102321304 576.5714285714284,23.116618075801746 577.2653061224489,23.87262960161157 577.9591836734693,24.654803695739016 578.6530612244898,25.462171374172325 579.3469387755101,26.293763652899724 580.0408163265306,27.148611547909454 580.734693877551,28.02574607518976 581.4285714285714,28.92419825072886 582.1224489795918,29.842999090515008 582.8163265306123,30.78117961053642 583.5102040816327,31.737770826781354 584.2040816326529,32.71180375523804 584.8979591836735,33.7023094118947 585.5918367346939,34.70831881273959 586.2857142857142,35.728862973760926 586.9795918367347,36.762972910946964 587.6734693877552,37.809679640285935 588.3673469387754,38.868014177766064 589.0612244897959,39.93700753937559 589.7551020408164,41.015690741102766 590.4489795918367,42.103094798935814 591.1428571428571,43.19825072886297 591.8367346938776,44.300189546872474 592.530612244898,45.40794226895255 593.2244897959183,46.52053991109146 593.9183673469388,47.63701348927743 594.6122448979593,48.75639401949867 595.3061224489795,49.877712517743454 596.0,51.0" stroke="#97A0BA" stroke-width="1" /> is_location_of <polyline fill="none" points="616.0,55.0 616.7346938775509,53.79607986468223 617.4693877551019,52.593128713376224 618.2040816326531,51.39211553009376 618.938775510204

In [0]:
re_html_2 = re_visualiser.display(re_results[1], relation_col='re_oncology_size_wip', return_html=True)

displayHTML(re_html_2)

The patient presented a 2 cm TUMOR_SIZE mass TUMOR_FINDING in her left breast, and the tumor TUMOR_FINDING in her other breast was 3 cm TUMOR_SIZE long. <polyline fill="none" points="294.0,55.0 294.59183673469386,53.79607986468223 295.1836734693877,52.593128713376224 295.7755102040817,51.39211553009376 296.36734693877554,50.194009298846574 296.95918367346934,48.999779003646445 297.55102040816325,47.810393628505125 298.1428571428573,46.626822157434425 298.7346938775511,45.450033574446024 299.32653061224494,44.280996863551756 299.9183673469387,43.12068100876335 300.5102040816326,41.97005499409259 301.1020408163265,40.83008780355124 301.6938775510204,39.70174842115105 302.2857142857143,38.586005830903794 302.8775510204082,37.48382901682123 303.469387755102,36.396186962915124 304.06122448979596,35.32404865319723 304.6530612244898,34.26838307167932 305.2448979591837,33.23015920237316 305.8367346938776,32.21034602929051 306.42857142857144,31.209912536443145 307.0204081632653,30.229827707842823 307.6122448979592,29.271060527501298 308.2040816326531,28.334579979430348 308.795918367347,27.421355047641715 309.3877551020408,26.532354716147186 309.9795918367347,25.668547968958514 310.57142857142867,24.83090379008747 311.16326530612247,24.020391163545803 311.7551020408163,23.237979073345294 312.34693877551024,22.484636503497697 312.9387755102041,21.761332438014776 313.53061224489795,21.069035860908297 314.12244897959187,20.408715756190023 314.7142857142858,19.781341107871725 315.30612244897964,19.187880899965155 315.8979591836735,18.629304116482082 316.48979591836735,18.106579741434267 317.08163265306126,17.620676758833483 317.6734693877551,17.172564152691482 318.265306122449,16.76321090702004 318.8571428571429,16.393586005830905 319.44897959183675,16.064658433135854 320.0408163265306,15.77739717294665 320.6326530612245,15.53277120927505 321.2244897959184,15.33174952613282 321.8163265306123,15.175301107531727 322.40816326530614,15.064394937483533 323.0,15.0 323.0,15.0 323.59183673469386,14.982762284422305 324.1836734693877,15.012035801409274 324.7755102040817,15.086851566949145 325.3673469387755,15.20624059703015 325.9591836734694,15.369233907640522 326.5510204081632,15.574862514768505 327.14285714285717,15.822157434402333 327.734693877551,16.11014968253024 328.32653061224494,16.43787027514046 328.91836734693874,16.80435022822123 329.5102040816326,17.208620557760792 330.10204081632645,17.64971227974738 330.6938775510204,18.126656410169232 331.2857142857143,18.638483965014576 331.8775510204082,19.184225960271654 332.46938775510205,19.762913411928704 333.06122448979596,20.37357733597396 333.6530612244898,21.01524874839565 334.2448979591837,21.686958665182026 334.8367346938774,22.387738102321304 335.42857142857133,23.116618075801746 336.0204081632653,23.87262960161157 336.61224489795916,24.654803695739016 337.204081632653,25.462171374172325 337.79591836734687,26.293763652899724 338.3877551020408,27.148611547909454 338.9795918367347,28.02574607518976 339.5714285714286,28.92419825072886 340.1632653061224,29.842999090515008 340.75510204081627,30.78117961053642 341.34693877551024,31.737770826781354 341.93877551020404,32.71180375523804 342.53061224489795,33.7023094118947 343.1224489795918,34.70831881273959 343.71428571428567,35.728862973760926 344.3061224489796,36.762972910946964 344.8979591836735,37.809679640285935 345.48979591836735,38.868014177766064 346.0816326530612,39.93700753937559 346.67346938775506,41.015690741102766 347.265306122449,42.103094798935814 347.8571428571429,43.19825072886297 348.4489795918367,44.300189546872474 349.0408163265306,45.40794226895255 349.63265306122446,46.52053991109146 350.2244897959184,47.63701348927743 350.81632653061223,48.75639401949867 351.40816326530614,49.877712517743454 352.0,51.0" stroke="#AB5E15" stroke-width="1" /> is_size_of <polyline fill="none" points="686.0,90.0 679.8673469387755,90.66326530612244 673.734693877551,91.32653061224487 667.6020408163266,91.98979591836735 661.4693877551022,92.653061224

In [0]:
re_html_3 = re_visualiser.display(re_results[2], relation_col='re_oncology_biomarker_result_wip', return_html=True)

displayHTML(re_html_3)

Immunohistochemical staining showed positive BIOMARKER_RESULT staining for CK5/6 BIOMARKER , P40 BIOMARKER and PD-L1 BIOMARKER , and negative BIOMARKER_RESULT staining for TTF-1 BIOMARKER , PD-1 BIOMARKER and weakly positive BIOMARKER_RESULT staining for ALK BIOMARKER . Immunohistochemistry study showed that the tumor cells are positive BIOMARKER_RESULT for epithelial markers-cytokeratin BIOMARKER and myoepithelial markers BIOMARKER , including cytokeratin 5/6 BIOMARKER , p63 BIOMARKER , and S100 BIOMARKER stains. <polyline fill="none" points="425.0,55.0 427.08163265306115,53.79607986468223 429.16326530612236,52.593128713376224 431.24489795918373,51.39211553009376 433.3265306122449,50.194009298846574 435.40816326530614,48.999779003646445 437.4897959183673,47.810393628505125 439.5714285714286,46.626822157434425 441.6530612244898,45.450033574446024 443.7346938775511,44.280996863551756 445.81632653061223,43.12068100876335 447.8979591836734,41.97005499409259 449.9795918367346,40.83008780355124 452.06122448979585,39.70174842115105 454.14285714285705,38.586005830903794 456.2244897959184,37.48382901682123 458.30612244897964,36.396186962915124 460.3877551020409,35.32404865319723 462.46938775510205,34.26838307167932 464.55102040816325,33.23015920237316 466.6326530612244,32.21034602929051 468.7142857142857,31.209912536443145 470.7959183673469,30.229827707842823 472.87755102040813,29.271060527501298 474.9591836734694,28.334579979430348 477.0408163265306,27.421355047641715 479.1224489795918,26.532354716147186 481.2040816326531,25.668547968958514 483.2857142857143,24.83090379008747 485.36734693877554,24.020391163545803 487.44897959183675,23.237979073345294 489.530612244898,22.484636503497697 491.61224489795916,21.761332438014776 493.6938775510204,21.069035860908297 495.7755102040816,20.408715756190023 497.8571428571429,19.781341107871725 499.9387755102041,19.187880899965155 502.0204081632653,18.629304116482082 504.1020408163265,18.106579741434267 506.18367346938777,17.620676758833483 508.265306122449,17.172564152691482 510.3469387755102,16.76321090702004 512.4285714285714,16.393586005830905 514.5102040816327,16.064658433135854 516.5918367346939,15.77739717294665 518.6734693877551,15.53277120927505 520.7551020408164,15.33174952613282 522.8367346938776,15.175301107531727 524.9183673469388,15.064394937483533 527.0,15.0 527.0,15.0 529.0816326530611,14.982762284422305 531.1632653061224,15.012035801409274 533.2448979591837,15.086851566949145 535.3265306122449,15.20624059703015 537.4081632653061,15.369233907640522 539.4897959183673,15.574862514768505 541.5714285714287,15.822157434402333 543.6530612244898,16.11014968253024 545.7346938775511,16.43787027514046 547.8163265306122,16.80435022822123 549.8979591836734,17.208620557760792 551.9795918367346,17.64971227974738 554.0612244897959,18.126656410169232 556.1428571428571,18.638483965014576 558.2244897959183,19.184225960271654 560.3061224489796,19.762913411928704 562.3877551020408,20.37357733597396 564.4693877551019,21.01524874839565 566.5510204081633,21.686958665182026 568.6326530612245,22.387738102321304 570.7142857142857,23.116618075801746 572.7959183673469,23.87262960161157 574.8775510204082,24.654803695739016 576.9591836734694,25.462171374172325 579.0408163265307,26.293763652899724 581.1224489795918,27.148611547909454 583.204081632653,28.02574607518976 585.2857142857143,28.92419825072886 587.3673469387755,29.842999090515008 589.4489795918366,30.78117961053642 591.5306122448981,31.737770826781354 593.6122448979592,32.71180375523804 595.6938775510205,33.7023094118947 597.7755102040817,34.70831881273959 599.8571428571429,35.728862973760926 601.9387755102041,36.762972910946964 604.0204081632653,37.809679640285935 606.1020408163265,38.868014177766064 608.1836734693877,39.93700753937559 610.2653061224489,41.015690741102766 612.3469387755102,42.103094798935814 614.4285714285714,43.19825072886297 616.5102040816327,44.300189546872474 618.5918367346939,45.40794226895255 620.6734693877551,46.52053991109146

## Assertion Status Models

With assertion status models, you will be able to identify if entities included in texts are mentioned as something present, absent, hypothetical, possible, etc. You can either try using the general assertion_oncology_wip model, or other models that are recommended for specific entity groups (such as assertion_oncology_problem_wip, which should be used for problem entities like Cancer_Dx or Metastasis).

In [0]:
assertion_oncology_wip = AssertionDLModel.pretrained("assertion_oncology_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_wip")

assertion_oncology_problem_wip = AssertionDLModel.pretrained("assertion_oncology_problem_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_tnm_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_problem_wip")

assertion_oncology_treatment_binary_wip = AssertionDLModel.pretrained("assertion_oncology_treatment_binary_wip", "en", "clinical/models") \
    .setInputCols(["sentence", 'ner_oncology_chunk', "embeddings"]) \
    .setOutputCol("assertion_oncology_treatment_binary_wip")

assertion_stages = ner_stages + [assertion_oncology_wip, assertion_oncology_problem_wip, assertion_oncology_treatment_binary_wip]

assertion_pipeline = Pipeline(stages=assertion_stages)

assertion_model = assertion_pipeline.fit(empty_data)

assertion_oncology_wip download started this may take some time.
[OK!]
assertion_oncology_problem_wip download started this may take some time.
[OK!]
assertion_oncology_treatment_binary_wip download started this may take some time.
[OK!]


In [0]:
sample_text_7 = 'The patient is suspected to have colorectal cancer. Family history is positive for other cancers. The result of the biopsy was positive. A CT scan was ordered to rule out metastases.'

sample_text_8 = 'The patient was diagnosed with breast cancer. She was suspected to have metastases in her lungs. Her family history is positive for ovarian cancer.'

sample_text_9 = 'The patient underwent a mastectomy. We recommend to start radiotherapy. The patient refused to chemotherapy.'

In [0]:
assertion_data = spark.createDataFrame(pd.DataFrame([sample_text_7, sample_text_8, sample_text_9], columns = ['text']))

In [0]:
assertion_results = assertion_model.transform(assertion_data).collect()

In [0]:
from sparknlp_display import AssertionVisualizer

assertion_visualiser = AssertionVisualizer()

In [0]:
assert_html_1 = assertion_visualiser.display(assertion_results[0], label_col ='ner_oncology_chunk', assertion_col='assertion_oncology_wip', return_html=True)

displayHTML(assert_html_1)

The patient is suspected to have colorectal cancer Cancer_Dx Possible . Family history is positive for other cancers Cancer_Dx Family . The result of the biopsy Pathology_Test Past was positive Pathology_Result Past . A CT scan Imaging_Test Past was ordered to rule out metastases Metastasis Possible .

In [0]:
assert_html_2 = assertion_visualiser.display(assertion_results[1], label_col ='ner_oncology_tnm_chunk', assertion_col='assertion_oncology_problem_wip', return_html=True)

displayHTML(assert_html_2)

The patient was diagnosed with breast cancer Cancer_Dx Medical_History . She was suspected to have metastases Metastasis Possible in her lungs. Her family history is positive for ovarian cancer Cancer_Dx Family_History .

In [0]:
assert_html_3 = assertion_visualiser.display(assertion_results[2], label_col ='ner_oncology_chunk', assertion_col='assertion_oncology_treatment_binary_wip', return_html=True)

displayHTML(assert_html_3)

The patient underwent a mastectomy Cancer_Surgery Present_Or_Past . We recommend to start radiotherapy Radiotherapy Hypothetical_Or_Absent . The patient refused to chemotherapy Chemotherapy Hypothetical_Or_Absent .

## Pretrained NER Profiling Pipelines

We can use pretrained NER profiling pipelines for exploring all the available pretrained NER models at once.

- `ner_profiling_oncology` : Returns results for oncology NER models.

For more examples, please check [this notebook](https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/11.2.Pretrained_NER_Profiling_Pipelines.ipynb).

<center><b>NER Profiling Oncology Model List</b>

|| | | |
|--------------|-----------------|-----------------|-----------------|
| ner_oncology_unspecific_posology | ner_oncology_tnm | ner_oncology_therapy | ner_oncology_test |
| ner_oncology_response_to_treatment | ner_oncology_posology | ner_oncology | ner_oncology_limited_80p_for_benchmarks |
| ner_oncology_diagnosis | ner_oncology_demographics | ner_oncology_biomarker | ner_oncology_anatomy_granular | ner_oncology_anatomy_general |



</center>

In [0]:
from sparknlp.pretrained import PretrainedPipeline

oncology_profiling_pipeline = PretrainedPipeline("ner_profiling_oncology", "en", "clinical/models")

ner_profiling_oncology download started this may take some time.
Approx size to download 2 GB
[OK!]


In [0]:
text = """The had previously undergone a left mastectomy and an axillary lymph node dissection for a left breast cancer twenty years ago.
The tumor was positive for ER and PR. Postoperatively, radiotherapy was administered to the residual breast.
The cancer recurred as a right lung metastasis 13 years later. He underwent a regimen consisting of adriamycin (60 mg/m2) and cyclophosphamide (600 mg/m2) over six courses, as first line therapy."""

In [0]:
oncology_result = oncology_profiling_pipeline.fullAnnotate(text)[0]
oncology_result.keys()

dict_keys(['ner_chunk_oncology_limited_80p_for_benchmarks', 'ner_oncology', 'ner_chunk_oncology_anatomy_general', 'document', 'ner_chunk_oncology_test', 'ner_chunk_oncology_tnm', 'ner_chunk_oncology', 'ner_chunk_oncology_therapy', 'ner_oncology_anatomy_general', 'ner_oncology_biomarker', 'ner_chunk_oncology_biomarker', 'ner_oncology_test', 'ner_oncology_response_to_treatment', 'token', 'ner_oncology_anatomy_granular', 'ner_oncology_therapy', 'ner_chunk_oncology_demographics', 'ner_chunk_oncology_response_to_treatment', 'ner_oncology_demographics', 'ner_oncology_diagnosis', 'ner_chunk_oncology_posology', 'ner_oncology_limited_80p_for_benchmarks', 'embeddings', 'ner_oncology_tnm', 'ner_chunk_oncology_diagnosis', 'ner_oncology_unspecific_posology', 'sentence', 'ner_chunk_oncology_unspecific_posology', 'ner_oncology_posology', 'ner_chunk_oncology_anatomy_granular'])

In [0]:
def get_token_results(light_result):

    tokens = [j.result for j in light_result["token"]]
    sentences = [j.metadata["sentence"] for j in light_result["token"]]
    begins = [j.begin for j in light_result["token"]]
    ends = [j.end for j in light_result["token"]]
    model_list = [ a for a in light_result.keys() if (a not in ["sentence", "token"] and "_chunks" not in a)]

    df = pd.DataFrame({'sentence':sentences, 'begin': begins, 'end': ends, 'token':tokens})

    for model_name in model_list:

        temp_df = pd.DataFrame(light_result[model_name])
        temp_df["jsl_label"] = temp_df.iloc[:,0].apply(lambda x : x.result)
        temp_df = temp_df[["jsl_label"]]

        # temp_df = get_ner_result(model_name)
        temp_df.columns = [model_name]
        df = pd.concat([df, temp_df], axis=1)

    # Filter columns to include only sentence, begin, end, token and all columns that start with 'ner_vop'
    filtered_df = df.loc[:, ['sentence', 'begin', 'end', 'token'] + [col for col in df.columns if col.startswith('ner_oncology')]]

    return filtered_df

In [0]:
get_token_results(oncology_result)

,sentence,begin,end,token,ner_oncology,ner_oncology_anatomy_general,ner_oncology_biomarker,ner_oncology_test,ner_oncology_response_to_treatment,ner_oncology_anatomy_granular,ner_oncology_therapy,ner_oncology_demographics,ner_oncology_diagnosis,ner_oncology_limited_80p_for_benchmarks,ner_oncology_tnm,ner_oncology_unspecific_posology,ner_oncology_posology
0,0,0,2,The,O,O,O,O,O,O,O,O,O,O,O,O,O
1,0,4,6,had,O,O,O,O,O,O,O,O,O,O,O,O,O
2,0,8,17,previously,O,O,O,O,O,O,O,O,O,O,O,O,O
3,0,19,27,undergone,O,O,O,O,O,O,O,O,O,O,O,O,O
4,0,29,29,a,O,O,O,O,O,O,O,O,O,O,O,O,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,3,410,411,as,O,O,O,O,O,O,O,O,O,O,O,O,O
75,3,413,417,first,B-Line_Of_Therapy,O,O,O,B-Line_Of_Therapy,O,B-Line_Of_Therapy,O,O,B-Line_Of_Therapy,O,O,O
76,3,419,422,line,I-Line_Of_Therapy,O,O,O,I-Line_Of_Therapy,O,I-Line_Of_Therapy,O,O,I-Line_Of_Therapy,O,O,O
77,3,424,430,therapy,O,O,O,O,O,O,O,O,O,O,O,O,O


## General Oncology Pretrained Pipelines

**`oncology pretrained ` Model List**

This pipeline includes Named-Entity Recognition, Assertion Status, Relation Extraction and Entity Resolution models to extract information from oncology texts.


|index|model|index|model|
|-----:|:-----|-----:|:-----|
| 1| [oncology_biomarker_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_biomarker_pipeline_en.html)  | 2| [oncology_general_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_general_pipeline_en.html)  |
| 3| [oncology_therapy_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_therapy_pipeline_en.html)  | 4| [oncology_diagnosis_pipeline](https://nlp.johnsnowlabs.com/2022/12/01/oncology_diagnosis_pipeline_en.html)

In [0]:
oncology_pipeline = PretrainedPipeline("oncology_biomarker_pipeline", "en", "clinical/models")

oncology_biomarker_pipeline download started this may take some time.
Approx size to download 1.6 GB
[OK!]


In [0]:
oncology_pipeline.model.stages

[DocumentAssembler_dab6eeac879e,
 SentenceDetectorDLModel_6bafc4746ea5,
 REGEX_TOKENIZER_9d695d8e04cc,
 WORD_EMBEDDINGS_MODEL_9004b1d00302,
 MedicalNerModel_ecf280ca65e5,
 NerConverter_929f666beebc,
 MedicalNerModel_aeadb24f76a3,
 NerConverter_4f9b8da8c4c3,
 MedicalNerModel_eb9da4b9039b,
 NerConverter_40028785be7b,
 MedicalNerModel_299a97740594,
 NerConverter_a8bf552d0249,
 MERGE_acda18b976a6,
 MERGE_12af4df6fa27,
 ASSERTION_DL_8d77f383c928,
 ASSERTION_DL_163867728788,
 POS_6f55785005bf,
 dependency_e7755462ba78,
 RelationExtractionModel_d0af74510daa,
 RelationExtractionModel_68ebe11369b6,
 RelationExtractionModel_513eb6317779]

In [0]:
text = """Immunohistochemistry was negative for thyroid transcription factor-1 and napsin A. The test was positive for ER and PR, and negative for HER2."""

result = oncology_pipeline.fullAnnotate(text)[0]

result.keys()

dict_keys(['re_oncology_granular_wip', 'assertion_oncology_test_binary_wip', 're_oncology_wip', 'ner_oncology_biomarker_wip_chunk', 'ner_biomarker_chunk', 'ner_oncology_test_wip', 'document', 're_oncology_biomarker_result_wip', 'merged_chunk', 'ner_oncology_biomarker_wip', 'ner_biomarker', 'ner_oncology_test_wip_chunk', 'token', 'embeddings', 'pos_tags', 'assertion_oncology_wip', 'assertion_chunk', 'ner_oncology_wip', 'dependencies', 'ner_oncology_wip_chunk', 'sentence'])

**NER Results**

In [0]:
chunks=[]
entities=[]
begins=[]
ends=[]
confidence=[]
for n in result['merged_chunk']:

    chunks.append(n.result)
    begins.append(n.begin)
    ends.append(n.end)
    entities.append(n.metadata['entity'])
    confidence.append(n.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'begin':begins, 'end':ends, 'entities':entities, 'confidence':confidence})

df

,chunks,begin,end,entities,confidence
0,Immunohistochemistry,0,19,Pathology_Test,0.9967
1,negative,25,32,Biomarker_Result,0.8323
2,thyroid transcription factor-1,38,67,Biomarker,0.296675
3,napsin A,73,80,Biomarker,0.64309996
4,positive,96,103,Biomarker_Result,0.8017
5,ER,109,110,Biomarker,0.948
6,PR,116,117,Biomarker,0.8711
7,negative,124,131,Biomarker_Result,0.8385
8,HER2,137,140,Oncogene,0.9359


**Assertion Status Results**

In [0]:
chunks=[]
entities=[]
status=[]
confidence=[]

for n,m in zip(result['merged_chunk'],result['assertion_oncology_test_binary_wip']):

    chunks.append(n.result)
    entities.append(n.metadata['entity'])
    status.append(m.result)
    confidence.append(m.metadata['confidence'])

df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status, 'confidence':confidence})

df

,chunks,entities,assertion,confidence
0,Immunohistochemistry,Pathology_Test,Medical_History,0.9926
1,negative,Biomarker_Result,Medical_History,0.9951
2,thyroid transcription factor-1,Biomarker,Medical_History,0.9951
3,napsin A,Biomarker,Medical_History,0.9926
4,positive,Biomarker_Result,Medical_History,0.9931
5,ER,Biomarker,Medical_History,0.9938


**Relation Extraction Results**

In [0]:
import pandas as pd

def get_relations_df (results, col='relations'):
  rel_pairs=[]
  for rel in results[0][col]:
      rel_pairs.append((
          rel.result,
          rel.metadata['entity1'],
          rel.metadata['entity1_begin'],
          rel.metadata['entity1_end'],
          rel.metadata['chunk1'],
          rel.metadata['entity2'],
          rel.metadata['entity2_begin'],
          rel.metadata['entity2_end'],
          rel.metadata['chunk2'],
          rel.metadata['confidence']
      ))

  rel_df = pd.DataFrame(rel_pairs, columns=['relation','entity1','entity1_begin','entity1_end','chunk1','entity2','entity2_begin','entity2_end','chunk2', 'confidence'])

  rel_df.confidence = rel_df.confidence.astype(float)

  return rel_df

In [0]:
result = oncology_pipeline.fullAnnotate(text)

rel_df = get_relations_df(result, 're_oncology_wip')

rel_df[rel_df.relation!= "O"]

,relation,entity1,entity1_begin,entity1_end,chunk1,entity2,entity2_begin,entity2_end,chunk2,confidence
1,is_related_to,Biomarker_Result,25,32,negative,Biomarker,38,67,thyroid transcription factor-1,0.997901
2,is_related_to,Biomarker_Result,25,32,negative,Biomarker,73,80,napsin A,0.999566
3,is_related_to,Biomarker_Result,96,103,positive,Biomarker,109,110,ER,0.987820
4,is_related_to,Biomarker_Result,96,103,positive,Biomarker,116,117,PR,0.897783
8,is_related_to,Biomarker_Result,124,131,negative,Oncogene,137,140,HER2,0.986855


In [0]:
result[0]['re_oncology_wip']

[Annotation(category, 0, 32, O, {'chunk2': 'negative', 'confidence': '0.97084755', 'entity2_end': '32', 'chunk1': 'Immunohistochemistry', 'entity1': 'Pathology_Test', 'entity2_begin': '25', 'chunk2_confidence': '0.8323', 'entity1_begin': '0', 'sentence': '0', 'direction': 'both', 'entity1_end': '19', 'entity2': 'Biomarker_Result', 'chunk1_confidence': '0.9967'}, []),
 Annotation(category, 25, 67, is_related_to, {'chunk2': 'thyroid transcription factor-1', 'confidence': '0.99790084', 'entity2_end': '67', 'chunk1': 'negative', 'entity1': 'Biomarker_Result', 'entity2_begin': '38', 'chunk2_confidence': '0.296675', 'entity1_begin': '25', 'sentence': '0', 'direction': 'both', 'entity1_end': '32', 'entity2': 'Biomarker', 'chunk1_confidence': '0.8323'}, []),
 Annotation(category, 25, 80, is_related_to, {'chunk2': 'napsin A', 'confidence': '0.9995658', 'entity2_end': '80', 'chunk1': 'negative', 'entity1': 'Biomarker_Result', 'entity2_begin': '73', 'chunk2_confidence': '0.64309996', 'entity1_beg